## Building a Chat Bot

In the examples below, we will build a very simple application that creates a simple chat bot. The bot is monitoring the actions that are happening on Slack, and posts some acknowledgement messages every time someone types a message.


#### Authentication

We start first by authenticating our application. (See notes in S1). We read the `slack_secret.json` file, and we instantiate then our SlackClient.

In [4]:
# Read the access token from the file
import json

secrets_file = 'slack_secret.json'
f = open(secrets_file, 'r') 
content = f.read()
f.close()

auth_info = json.loads(content)
token = auth_info["access_token"]

In [5]:
# If you happen not to have the slackclient library installed, 
# you can uncomment and run the code below
!sudo -H python3 -m pip install -U slackclient

Requirement already up-to-date: slackclient in /usr/local/lib/python3.5/dist-packages
Requirement already up-to-date: websocket-client in /usr/local/lib/python3.5/dist-packages (from slackclient)
Requirement already up-to-date: requests in /usr/local/lib/python3.5/dist-packages (from slackclient)
Requirement already up-to-date: six in /usr/lib/python3/dist-packages (from slackclient)


In [7]:
from slackclient import SlackClient
sc = SlackClient(token)
response = sc.api_call("users.info", user=auth_info["user_id"])
user = response['user']
print("The username of the authenticated user is", user.get('name'))
print("The email of the authenticated user is", user.get('profile').get('email'))
print("The email of the authenticated user is", user.get('profile').get('real_name'))

The username of the authenticated user is panos
The email of the authenticated user is panos@stern.nyu.edu
The email of the authenticated user is Panos Ipeirotis


### Implementing the bot

The code below continuously monitors the various events on Slack. Of course, the bot can be programmed to react to any type of event. In this particular example, the bot monitors for user messages, and posts a "thank you" note to each user into the #bots channel.

In [8]:
import time

if sc.rtm_connect():
    # We are going to be polling the Slack API for recent events continuously
    while True:
        # We are going to wait 1 second between monitoring attempts
        time.sleep(1)
        # If there are any new events, we will get a response. If there are no events, the response will be empty
        response = sc.rtm_read()
        for item in response:
            # We print the event on the screen for informational purposes
            print(item) 
            event_type = item.get("type")
            if event_type == 'message' and item.get("user")!=None:
                print(item)
                message = "Thank you user {u} for participating in channel {c}".format(u=item["user"], c=item["channel"])
                sc.api_call("chat.postMessage", channel="#bots", text=message)

{'type': 'hello'}
{'reply_to': 41927, 'channel': 'C44A5QZ6F', 'text': 'you need to have a primary key to be different between two different rows', 'type': 'message', 'user': 'U3VAU7WF4', 'ts': '1487043809.000073'}
{'reply_to': 41927, 'channel': 'C44A5QZ6F', 'text': 'you need to have a primary key to be different between two different rows', 'type': 'message', 'user': 'U3VAU7WF4', 'ts': '1487043809.000073'}
{'url': 'wss://mpmulti-1rfk.slack-msgs.com/websocket/fYCsBslnTpyEp-BQLJ2t_NpL2h_TYa20udCuUtf8d9RLul1OnMLApo_d8QBmfkuHjRmPi4sF-kVmIuyGYPq3G7FAE8TeSUBB72JYQyApd7G8eQBG5wOSk8ekrc_ZUvkwg6Ux9dZxwkfYgLba1UYjF21ButhtW1jszAfL2yuMme4=', 'type': 'reconnect_url'}
{'presence': 'active', 'user': 'U3VAU7WF4', 'type': 'presence_change'}
{'bot_id': 'B44MCNJDV', 'team': 'T3W1D2Y3G', 'subtype': 'bot_message', 'text': 'Thank you user U3VAU7WF4 for participating in channel C44A5QZ6F', 'username': 'MagnificentPanoBot', 'ts': '1487046379.000882', 'type': 'message', 'channel': 'C429KQ60N'}
{'unread_count':

KeyboardInterrupt: 

#### Exercises 

* Change the code above to retrieve the actual user name of the user, instead of the user id
* Chagne the code above to retrieve the actual name of the channel instead of the channel id
* Change the code above to post the answer to the channel where the user posted, instead of the '#bots' channel